It's best if at this step you already have a list of paper titles for each track at VIS. These should be saved in InfoVisTitles.txt, SciVisTitles.txt, VASTTitles.txt. Then you download .json data into the respective new folder. 

Currently the api is broken so that you need two calls to get this data that might result in overlapped data. See API queries.txt

This needs to be read in and cleaned up. For 2019 I named the resulting json file: articles-preprint-1.json and articles-preprint-reverseSort.json

In [1]:
#now I read in the two json files and try to concatenate them

import pandas as pd

year_to_import = year = "2019"

files = ["articles-preprint-1.json","articles-preprint-reverseSort.json"]

filepaths = [year + "/" + files[0],year + "/" + files[1]]

final_columns = ['Conference', 'Year', 'Paper Title', 'Paper DOI', 'Link', 'First page','Last page','Paper type','Abstract', 'Author Names', 'Author Affiliation', 'References','Author Keywords','Citing papers - IEEE','Citing patents - IEEE']


In [2]:
##here is the same conversion code json-csv as used in Step 1 

import sys
import json
import csv

##
# Convert to string keeping encoding in mind...
##
def to_string(s):
    try:
        return str(s)
    except:
        #Change the encoding type if needed
        return s.encode('utf-8')


##
# This function converts an item like 
# {
#   "item_1":"value_11",
#   "item_2":"value_12",
#   "item_3":"value_13",
#   "item_4":["sub_value_14", "sub_value_15"],
#   "item_5":{
#       "sub_item_1":"sub_item_value_11",
#       "sub_item_2":["sub_item_value_12", "sub_item_value_13"]
#   }
# }
# To
# {
#   "node_item_1":"value_11",
#   "node_item_2":"value_12",
#   "node_item_3":"value_13",
#   "node_item_4_0":"sub_value_14", 
#   "node_item_4_1":"sub_value_15",
#   "node_item_5_sub_item_1":"sub_item_value_11",
#   "node_item_5_sub_item_2_0":"sub_item_value_12",
#   "node_item_5_sub_item_2_0":"sub_item_value_13"
# }
##
def reduce_item(key, value):
    global reduced_item
    
    #Reduction Condition 1
    if type(value) is list:
        i=0
        for sub_item in value:
            reduce_item(key+'_'+to_string(i), sub_item)
            i=i+1

    #Reduction Condition 2
    elif type(value) is dict:
        sub_keys = value.keys()
        for sub_key in sub_keys:
            reduce_item(key+'_'+to_string(sub_key), value[sub_key])
    
    #Base Condition
    else:
        reduced_item[to_string(key)] = to_string(value)

In [3]:
#now we read the json and convert to a csv

for path in filepaths:

        csv_file_path = path+".csv"
        node = "articles"
        
        #TODO: We could check if the .csv file already exists and then decide not to do anything

        fp = open(path, 'r',encoding="utf8")
        json_value = fp.read()
        raw_data = json.loads(json_value)

        try:
            data_to_be_processed = raw_data[node]
        except:
            data_to_be_processed = raw_data

        processed_data = []
        header = []
        for item in data_to_be_processed:
            reduced_item = {}
            reduce_item(node, item)

            header += reduced_item.keys()

            processed_data.append(reduced_item)

        header = list(set(header))
        header.sort()

        with open(csv_file_path, 'w+',encoding="utf8") as f:
            writer = csv.DictWriter(f, header, quoting=csv.QUOTE_ALL)
            writer.writeheader()
            for row in processed_data:
                writer.writerow(row)

        print ("Just completed writing "+csv_file_path+" file with %d columns" % len(header))


Just completed writing 2019/articles-preprint-1.json.csv file with 68 columns
Just completed writing 2019/articles-preprint-reverseSort.json.csv file with 84 columns


In [4]:
#now we have our csv files and we can reduce them to the data we need
#this code is copied from step 2

from pathlib import Path

full_df = None

#lets concatenate all json files

for jsonpath in filepaths:
    path = jsonpath + ".csv"
    
    df = pd.read_csv(path,dtype=object)
    #print(df.head())
    
    if full_df is None:
        full_df = df
    else:
        full_df = full_df.append(df, ignore_index = True)
    print(str(len(full_df.index)))
        
        
#and remove duplicate rows
full_df.drop_duplicates(subset ="articles_doi",inplace = True) 
xplore_df = full_df

xplore_df.to_csv(year_to_import+"/Vispubdata-preprint-unconverted.csv",index =False)


#get all column names
columns = xplore_df.columns
#print(columns)

#remove all the columns we don't need
#-------------------------------------------------------------
xplore_df.drop('articles_access_type', axis=1, inplace=True,errors='ignore')
xplore_df.drop('articles_abstract_url', axis=1, inplace=True,errors='ignore')
xplore_df.drop('articles_publisher', axis=1, inplace=True,errors='ignore')
xplore_df.drop('articles_article_number', axis=1, inplace=True,errors='ignore')
xplore_df.drop('articles_volume', axis=1, inplace=True,errors='ignore')
xplore_df.drop('articles_rank', axis=1, inplace=True,errors='ignore')
xplore_df.drop('articles_publication_number', axis=1, inplace=True,errors='ignore')
xplore_df.drop('articles_publication_date', axis=1, inplace=True,errors='ignore')
xplore_df.drop('articles_pdf_url', axis=1, inplace=True,errors='ignore')
xplore_df.drop('articles_issue', axis=1, inplace=True,errors='ignore')
xplore_df.drop('articles_issn', axis=1, inplace=True,errors='ignore')
xplore_df.drop('articles_is_number', axis=1, inplace=True,errors='ignore')
xplore_df.drop('articles_html_url', axis=1, inplace=True,errors='ignore')
xplore_df.drop('articles_publication_title', axis=1, inplace=True,errors='ignore')



#WARNING: I assume the file is correctly ordered. If not we need to do something more fancy
columns_to_drop = columns[columns.str.contains("author_order")] 
xplore_df.drop(columns_to_drop, axis=1, inplace=True)

columns_to_drop = columns[columns.str.contains("ieee_terms")] 
xplore_df.drop(columns_to_drop, axis=1, inplace=True)

#rename the columns we want to keep
xplore_df.rename(index=str, inplace=True, columns={"articles_title":"Paper Title","articles_start_page":"First page","articles_abstract":"Abstract","articles_publication_year": "Year", "articles_content_type": "Paper type","articles_end_page":"Last page","articles_doi":"Paper DOI","articles_citing_patent_count":"Citing patents - IEEE","articles_citing_paper_count":"Citing papers - IEEE"})



#put all author full names together
#----------------------------------------
author_columns = columns[columns.str.contains("full_name")]
xplore_df[author_columns].fillna(value="")
authors = xplore_df[author_columns].apply(lambda x: ';'.join(x.dropna().values.tolist()), axis=1)
#we have the authors put together, now add them to the df
xplore_df['Author Names'] = authors
#now remove all the individual columns that we no longer need
xplore_df.drop(author_columns, axis=1, inplace=True)

#put all affiliations together
#------------------------------------------
affiliation_columns = columns[columns.str.contains("affiliation")]
xplore_df[affiliation_columns].fillna(value="")
affiliations = xplore_df[affiliation_columns].apply(lambda x: ';'.join(x.dropna().values.tolist()), axis=1)
#we have the affiliations put together, now add them to the df
xplore_df['Author Affiliation'] = affiliations
#now remove all the individual columns that we no longer need
xplore_df.drop(affiliation_columns, axis=1, inplace=True)

#put all author keywords
#------------------------------------------
kw_columns = columns[columns.str.contains("author_terms")]
xplore_df[kw_columns].fillna(value="")
keywords = xplore_df[kw_columns].apply(lambda x: ','.join(x.dropna().values.tolist()), axis=1)
#we have the keywords put together, now add them to the df
xplore_df['Author Keywords'] = keywords
#now remove all the individual columns that we no longer need
xplore_df.drop(kw_columns, axis=1, inplace=True)

#create the link column
#--------------------------------------------
xplore_df["Link"] = 'http://dx.doi.org/' + xplore_df["Paper DOI"]

#now add columns that are missing
xplore_df["Conference"] = ["conference_external"] * len(xplore_df.index)
xplore_df["References"] = [""] * len(xplore_df.index)


#now reorder the columns

xplore_df = xplore_df[final_columns]

#['Conference', 'Year', 'Paper Title', 'Paper DOI', 'Link', 'First page',
#       'Last page',
#       'Paper type: C=conference paper, J = journal paper, M=miscellaneous (capstone, keynote, VAST challenge, panel, poster, ...)',
#       'Abstract', 'Author Names', 'Author Affiliation', 'References',
#       'Author Keywords', 'Experimental: OCRd Author Affiliations']


#print(xplore_df.info())
#print(xplore_df.index)

conf_title_files=[year_to_import+'/VASTTitles.txt',year_to_import+'/InfoVisTitles.txt',year_to_import+'/SciVisTitles.txt']

titles = xplore_df['Paper Title'].str.lower()
titles = titles.str.strip()
titles_not_found = [ ] #making this a regular python list, simplifies appending
conf_not_found = [ ]

for conf in conf_title_files:
    if Path(conf).is_file():
        conference = "InfoVis"
        #print(conf)

        if "VAST" in conf: 
            conference = 'VAST'
        elif "SciVis" in conf:
            conference = 'SciVis'

        print("Extracted conference name: " + conference)

        #there is a title file, so go read it out
        conf_titles = pd.read_csv(conf,header=None,names=['Title'],sep=";")
        conf_titles = conf_titles['Title'].str.lower()
        conf_titles = conf_titles.str.strip()

        for t in conf_titles:

            #now check if the title exists in the read out data
            found = titles[titles == t].any() 
            if(found != False):
                rowindex = pd.Index(titles).get_loc(t)
                #print(rowindex)
                #print(xplore_df.loc[rowindex,0])
                xplore_df.at[str(rowindex),'Conference'] = conference
            else:
                titles_not_found.append(t)
                conf_not_found.append(conference)



    else:
        print("Title file: " + conf + " not found")

#next we remove all rows where the conference is neither SciVis, InfoVis, or VAST and we should have our IEEE VIS preprints
xplore_df = xplore_df[xplore_df['Conference'] != 'conference_external']

xplore_df.to_csv(year_to_import+"/Vispubdata-preprint.csv",index =False)
pd.DataFrame({'Conference':conf_not_found,'Title':titles_not_found}).to_csv(year_to_import+'/TitlesNotFound-Preprint.csv',index=False)
print("Done")



200
400


C:\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Extracted conference name: VAST
Extracted conference name: InfoVis
Extracted conference name: SciVis
Done


In [5]:
#now we need to dedupe authors

xplore_df = pd.read_csv(year_to_import+"/Vispubdata-preprint-cleaned.csv")

deduped_vispubdataentries = list(pd.read_csv("Deduped-Authors.csv")['Authors'])

non_deduped_authors = []

authors = xplore_df['Author Names'].dropna()


for author in authors:
    authors = author.split(';')
    for a in authors:
        non_deduped_authors.append(a)


    
non_deduped_unique_authors = list(set(non_deduped_authors))

#print(deduped_vispubdataentries)

print("Authors not found in the deduped set")

non_matched_authors = []

#i = 1;
for author in non_deduped_unique_authors:
    if not author in deduped_vispubdataentries:
        non_matched_authors.append(author)
        #print(str(i) + " " + author)
        #i = i + 1

print(non_matched_authors)
#for author in non_deduped_unique_authors:
    


Authors not found in the deduped set
['Chittayong Surakitbanharn', 'Micah Acinapura', 'Robert M. Kirby', 'Ying Yang', 'Khai N. Truong', 'Miriah Meyer', 'Viswanath Aluru', 'Geoffrey S. Young', 'Olivier Thonnard', 'Brian Lim', 'Liangge Hsu', 'Tobias Gänther', 'Vincent Bloemen', 'Casimir J.H. Ludwig', 'Ke Li', 'Bo Qiao', 'Doris Kosminsky', 'Jie Bao', 'Yongsu Ahn', "Nancy O'Brien", 'Leilani Battle', 'Tobias Isenberg', 'Ananda Montoly', 'Jinsong Wang', 'Zeyu Li', 'Elizabeth Munch', 'Sarkis Halladjian', 'Rushil Anirudh', 'Julian Thijssen', 'Jungu Choi', 'Yang Yue', 'Brian K. Spears', 'Ragini Rathore', 'Artem Sokolov', 'Krešimir Matković', 'Youssef S. G. Nashed', 'Wenyuan Wang', 'Stefan Rädisühli', 'Aoyu Wu', 'Adam J. Bradley', 'Ji Qi', 'Jorge A. Wagner Filho', 'Mahima Pushkarna', 'Harsh Shukla', 'Nicolas Roussel', 'Yu Dong', 'Natalia Andrienko', 'Yu Zhang', 'Jinson Zhang', 'Yue Li', 'Mingliang Xue', 'Shilpika Shilpika', 'Xiaotong Li', 'Joel Shapiro', 'Christian Frisson', 'Dongmei Zhang', 'La

In [15]:
import Levenshtein as lev
# Str1 = "Apple Inc."
# Str2 = "apple Inc"
# Distance = lev.distance(Str1.lower(),Str2.lower()),
# print(Distance)
# Ratio = lev.ratio(Str1.lower(),Str2.lower())
# print(Ratio)

#do some fuzzy string matching
#this could take a while

matchedauthors = []
levenshteinAlternative = []
doublemetaphoneAlternatives = []

from metaphone import doublemetaphone
from enum import Enum

class Threshold(Enum):
    WEAK = 0
    NORMAL = 1
    STRONG = 2
    
def double_metaphone(value):
    #print(doublemetaphone(value))
    return doublemetaphone(value)

#(Primary Key = Primary Key) = Strongest Match
#(Secondary Key = Primary Key) = Normal Match
#(Primary Key = Secondary Key) = Normal Match
#(Alternate Key = Alternate Key) = Minimal Match
def double_metaphone_compare(tuple1,tuple2,threshold):
    if threshold == Threshold.WEAK:
        if tuple1[1] == tuple2[1]:
            return True
    elif threshold == Threshold.NORMAL:
        if tuple1[0] == tuple2[1] or tuple1[1] == tuple2[0]:
            return True
    else:
        if tuple1[0] == tuple2[0]:
            return True
    return False

for author in non_matched_authors:
    ratio = 0
    bestmatchLev = ""
    bestmatchMet = ""
    matchedauthors.append(a)
    tp1 = double_metaphone(author)
    
    for deduped_author in deduped_vispubdataentries:
        #pure levenshtein comparison
        r = lev.ratio(author.lower(),deduped_author.lower())
        if r > ratio and r > 0.8:
            ratio = r
            bestmatchLev = deduped_author
        elif r == ratio and r > 0.8:
            bestmatchLev = bestmatchLev + ";" + deduped_author
            
        #double metaphone comparison:
        tp2 = double_metaphone(deduped_author)
        if double_metaphone_compare(tp1,tp2,Threshold.NORMAL):
            bestmatchMet = bestmatchMet+";"+deduped_author
        
            
    levenshteinAlternative.append(bestmatchLev)
    doublemetaphoneAlternatives.append(bestmatchMet)
    
    
    if bestmatchLev != ""  or bestmatchMet != "":
        print("best match for: " + author + " in DBLP is: " + bestmatchLev + " or " + bestmatchMet)
        

best match for: Ying Yang in DBLP is: Jing Yang;Yang Yang or ;Jing Yang
best match for: Miriah Meyer in DBLP is: Miriah D. Meyer or 
best match for: Tobias Gänther in DBLP is: Tobias Günther or 
best match for: Jie Bao in DBLP is: Jie Gao or 
best match for: Tobias Isenberg in DBLP is: Tobias Isenberg 0001 or 
best match for: Jinsong Wang in DBLP is: Xinsong Yang or 
best match for: Jungu Choi in DBLP is:  or ;Yingcai Wu
best match for: Yang Yue in DBLP is:  or ;Jing Hua;Jian Cui;Jiang Wu;Jing Ye;Jing Wu
best match for: Krešimir Matković in DBLP is: Kresimir Matkovic or 
best match for: Aoyu Wu in DBLP is:  or ;Jai Y. Yu
best match for: Ji Qi in DBLP is:  or ;Weiwei Cui;Yueqi Hu;Wei Qiao;Yi Gu
best match for: Natalia Andrienko in DBLP is: Natalia V. Andrienko or 
best match for: Yu Zhang in DBLP is: Yubo Zhang or ;Jiawei Zhang
best match for: Yue Li in DBLP is:  or ;Jie Li;Jie Liu;Jie Lu
best match for: Xiaotong Li in DBLP is: Xiaotong Liu or 
best match for: Christian Frisson in DBLP 

In [19]:
original = ["Robert M. Kirby","Miriah Meyer","Tobias Isenberg","Krešimir Matković","Natalia Andrienko","Lace M.K. Padilla","Charles Hansen","Fabian Beck","Gregory Abram","J. Fuchs","J. Kehrer","John Lee","Deokgun Park","John Stasko","Markus Gross","Huub Van De Wetering","Jens Krüger","Gennady Andrienko","David Kouřil","Boudewijn P.F. Lelieveldt","Jan Byška","Cláudio Silva","Denis Gračanin","Mingming Fan","Michelle A. Borkin","Robert Krueger","Fernanda Viégas","Jarke van Van Wijk","Carlos Scheidegger","Daniel Keim","D. A. Keim"]
replacement = ["Robert Michael Kirby","Miriah D. Meyer","Tobias Isenberg 0001","Kresimir Matkovic","Natalia V. Andrienko","Lace M. K. Padilla","Charles D. Hansen","Fabian G. Beck","Greg Abram","Johannes Fuchs","Johannes Kehrer","John A. Lee","Deok Gun Park","John T. Stasko","Markus H. Gross"," Huub van de Wetering","Jens H. Krüger","Gennady L. Andrienko","David Kouril","Boudewijn P. F. Lelieveldt","Jan Byska","Cláudio T. Silva","Denis Gracanin","Mingming Fan 0001","Michelle Borkin","Robert Krüger","Fernanda B. Viégas","Jarke J. van Wijk","Carlos Eduardo Scheidegger","Daniel A. Keim","Daniel A. Keim"]

hand_deduped_authors = []

for a in xplore_df['Author Names']:
    authors = a.split(';')
    deduped_authors = ""
    for singleauthor in authors:
        if singleauthor in original:
            index = original.index(singleauthor)
            singleauthor = replacement[index]
        deduped_authors = deduped_authors + ";" + singleauthor
    
    deduped_authors = deduped_authors[1:]
    hand_deduped_authors.append(deduped_authors)
    
xplore_df['AuthorNames-Deduped'] =  hand_deduped_authors

print(xplore_df.head())

xplore_df.to_csv(year+"_preprintData-hand-deduped.csv",index=False)
    

  Conference  Year                                        Paper Title  \
0     SciVis  2019  Progressive Wasserstein Barycenters of Persist...   
1     SciVis  2019  Scale-Space Splatting: Reforming Spacetime for...   
2     SciVis  2019  A Structural Average of Labeled Merge Trees fo...   
3     SciVis  2019  Toward Localized Topological Data Structures: ...   
4     SciVis  2019  Artifact-Based Rendering: Harnessing Natural a...   

                   Paper DOI                                         Link  \
0  10.1109/TVCG.2019.2934256  http://dx.doi.org/10.1109/TVCG.2019.2934256   
1  10.1109/TVCG.2019.2934258  http://dx.doi.org/10.1109/TVCG.2019.2934258   
2  10.1109/TVCG.2019.2934242  http://dx.doi.org/10.1109/TVCG.2019.2934242   
3  10.1109/TVCG.2019.2934257  http://dx.doi.org/10.1109/TVCG.2019.2934257   
4  10.1109/TVCG.2019.2934260  http://dx.doi.org/10.1109/TVCG.2019.2934260   

   First page  Last page             Paper type  \
0           1          1  Early Access Articles

In [60]:
"Alex Endert" == deduped_vispubdataentries[3267]

True

In [61]:
"Alex Endert" is deduped_vispubdataentries[3267]

False

In [68]:
non_deduped_unique_authors

['Alex Endert',
 'Hanyi Chen',
 'Jagoda Walny',
 'Gerik Scheuermann',
 'Chen Bao',
 'Tom Peterka',
 'Jiazhi Xia',
 'Christopher Collins',
 'D. A. Keim',
 'Kai Liu',
 'Chittayong Surakitbanharn',
 'Vanessa Peña-Araya',
 'B. Elgohari',
 'Haidong Zhang',
 'Denis Gračanin',
 'Carsten Dachsbacher',
 'Pavol Klacansky',
 'Baldur Van Lew',
 'Michael J. McGuffin',
 'Amir A. Zamani',
 'Helwig Hauser',
 'Audace Nakeshimana',
 'Wei Chen',
 'Karrie Karahalios',
 'Anna Vilanova',
 'Deokgun Park',
 'Harsh Shukla',
 'Bahador Saket',
 'Kaiyi Zhang',
 'Xiaoyu Zhang',
 'Robert Krüger',
 'Andreas Walch',
 'Michelle A. Borkin',
 'Wei Zeng',
 'Dirk Streeb',
 'Eugene Zhang',
 'Dongmei Zhang',
 'Olivier Thonnard',
 'Alexander M. Rush',
 'Fearn Bishop',
 'Susannah Burrows',
 'Han-Wei Shen',
 'Shahid Latif',
 'Yuanzhe Chen',
 'Brian D. Ondov',
 'Siming Chen',
 'Jungu Choi',
 'Zeyu Wang',
 'Cristina R. Ceja',
 'Shuai Chen',
 'Mingliang Xue',
 'Winter Wei',
 'Alexandra Diehl',
 'Ko-Chih Wang',
 'Adam Jurčík',
 'W

In [69]:
deduped_vispubdataentries

0             Chuck Woodbury
1         Alfie Abdul-Rahman
2           Emanuel Zgraggen
3             Eamonn Maguire
4        Jean-Marc Verbavatz
5       Tobias Isenberg 0001
6                Haiyan Yang
7            L. I. Ponomarev
8                Meg Pirrung
9             Fabian Sperrle
10        Mariappan S. Nadar
11        Jonathan Konieczny
12             Javier Bescos
13              Thomas Nocke
14             David M. Krum
15              Han Krishnan
16              Harsh Bhatia
17           Micah T. Taylor
18           Gerald L. Lohse
19             Marcus Handte
20           Graham J. Wills
21           Charles Aldrich
22              Chris Faigle
23              Tiago Etiene
24             Jorge Estrada
25                Wulue Zhao
26                Bob McGrew
27               Markus John
28              Boris Yamrom
29          Monica K. Hurdal
                ...         
5453          Sandra Bringay
5454           Gleb Bahmutov
5455          Jessie Kennedy
5456        Al